## Advanced String Manipulation with Regular Expressions

We have author contributions for several PNAS Journals. Howver, these author contributions are not in machine-readable format. An example of the data is like this: `<p>Author contributions: V.G. and A.P. designed research; V.G., C.F., M.C., V.L.B., M.M., and F.D.N. performed research; E.C., F.R., and L.F. contributed new reagents/analytic tools; A.M., S.S., and G.M. analyzed data; and A.P. wrote the paper.</p>`

How can we extract unique author contribution groups for each of the journals? We want to break down the data into unique groups such as `designed research`, `performed research` etc.

#### Reading our Data

In [1]:
import pandas as pd
import numpy as np
import regex as re

In [2]:
df = pd.read_csv('meta.csv')

In [3]:
df.head()

,VOL,ISS,DOI,Author contributions
0,119,10,10.1073/pnas.2104718119,<p>Author contributions: V.G. and A.P. designe...
1,119,10,10.1073/pnas.2105416119,<p>Author contributions: N.M. and M.v.d.P. des...
2,119,10,10.1073/pnas.2107357119,"<p>Author contributions: X.F., H.W., and J.G. ..."
3,119,10,10.1073/pnas.2107453119,<p>Author contributions: L.W. and W.Y. designe...
4,119,10,10.1073/pnas.2107720119,<p>Author contributions: Y.V.W. and R.R.S. des...


#### Understanding our data

To know the formatting options to use, you have to thoroughly understand your data. The series of code you'll see are the results after several iterations of code and writing to csv files to see how clean our results are. You have to be able to identify patterns in the text that you can then manipulate with regex. Different people will approach the problem in different ways. You should work iteratively, applying solutions step by step. Note that depending on the data, it might be impossible to get 100% clean data. You might have to extract to csv and still make corrections manually.

This is my approach to extracting the information that I need. Feel free to download this notebook and explore ways to get better results.

#### Conditions for formatting our data

- All rows starting from "Contributed by" does not contain the information we need. 
- All rows start with \<p> and end with \<\p>.
- All text starting with these variants contain the information we need: "Author Contribution", "Author Contributions","Authors Contributions", and "Author's Contributions"
- If text does not start with the variants above, it will start with author names, or it does not contain the information we need.
- Author names are of different formats: `A.B., A. Borden, A.v.d.B., A.{special characters, numbers}B., A.-B.`
- Author contributions always start with small letters. Any sentence starting with capital letter that is not Author Contributions or an initial does not contain the information we need.
- Majority of author contributions are separated by `;` e.g. (designed research; performed research), while some are separated by , e.g. (designed research, performed research)
- Majority of the last author contribution in a string is preceded by `, and` e.g. designed research, and performed research.

#### String Formatting and Regex

We start by taking out extraneous information.

First, we delete the \<p>, \<\p>, and author contributions variants that are start and end majority of the rows.

In [4]:
#version13 = df[~df['Author contributions'].str.contains('Contributed by')]

In [5]:
version14 = df['Author contributions'].str.replace('<p>Author contributions: ','')

In [6]:
version14 = version14.str.replace('<p>Author contribution: ','')

In [7]:
version14 = version14.str.replace('<p>Authors contributions: ','')

In [8]:
version14 = version14.str.replace('<p>Author"s contributions: ','')

In [9]:
version14.sample(5)

1400     Z.R., H.L., P.M.G., M.D., A.S.K., A.T.F., M.L....
46655    K.C.N., G.C., D.B., and V.C. designed research...
14996                          M.L.S. wrote the paper.</p>
19968    A.I., M.R.-S., and R.W. designed research, per...
7968     B.A.R. and C.C. designed research, performed r...
Name: Author contributions, dtype: object

In [10]:
version14 = version14.str.replace('<p>','')

In [11]:
version14 = version14.str.replace('</p>','')

Then, we filter out the rows that have the information that we want.

We do this by using regex match to filter for rows that start with initials i.e. a capital letter and period.

In [12]:
#version15 = version14.apply(lambda x: re.search('^[A-Z]\.',str(x)))
version15 = version14[version14.str.match('^[A-Z]\.')]

In [13]:
version15.head()

0    V.G. and A.P. designed research; V.G., C.F., M...
1    N.M. and M.v.d.P. designed research; N.M., H.P...
2    X.F., H.W., and J.G. designed research; X.F., ...
3    L.W. and W.Y. designed research; Y.C., D.G., Z...
4    Y.V.W. and R.R.S. designed research; Y.V.W. an...
Name: Author contributions, dtype: object

Now we can use different regex techniques to clean the rows and extract the information we want. The sample string below shows what each of the functions that we apply to the dataframe does. The string includes all of the different variations that we noticed in the data. You can see how it iteratively cleans the string and extracts the information we need. We then apply these same functions to our dataframe.

In [14]:
#sample string
t = 'Y.W., J. Cembrano, C.M.-S., A.R.W., M.&#x00D8;., Ã–. designed research, Å½.S. performed research; A.J.O. and J.D. developed the coating; R. Herbig contributed new reagents/analytic tools, and a.a.H. wrote the paper.'

#converts all lowercase intials (e.g. v.d.) to uppercase (V.D.).
#the negative lookahead (?!$) ensures that ending of strings are not affected.
callback = lambda pat: pat.group(1).upper()+'.'
a = re.sub('([a-z])\.(?!$)', callback, t)

#converts all , to ;
#ensures that we have a single separator ;
c = re.sub(',',';',a)

#deletes all author names
#deletes intials like A.B. and A. Borden
f = re.sub('[A-Z]\.|[A-Z]\w*','',c)

#deletes all text that is not alphabetic, ;, space or /
#we leave the ; because it is our separator
#we leave the space because deleting it can cause complications (test to see)
m = re.sub('[^a-zA-Z;\s]\w*','',f)

#we delete and because it precedes many author contributions e.g. designed research, and performed research
#this could cause confusion in the contributions
#e.g. analyzed and wrote the report becomes: analyzed wrote the report
d = re.sub('and',';',m) #change from `and ` to `; and`.

#we delete any leading/extraneous space that could have occured due to the deletions
n = d.lstrip()
r = re.sub('\\s+',' ',n)
#we delete recurring ; and ensure we have just one ; separating each contribution
#we also delete all . at the end of each row
q = re.sub('(;\s)+',';',r)
v = re.sub('\.$','',q)
y = re.sub('^;*','',v)

#you can look at each output and try to make the code more efficient i.e. reduce the number of lines

print(a)
#print(c)
#print(f)
print(m)
print(d)
#print(n)
#print(r)
#print(q)
#print(v)
print(y)

Y.W., J. Cembrano, C.M.-S., A.R.W., M.&#x00D8;., Ã–. designed research, Å½.S. performed research; A.J.O. and J.D. developed the coating; R. Herbig contributed new reagents/analytic tools, and A.A.H. wrote the paper.
;  ; ; ; ;;  designed research;  performed research;  and  developed the coating;   contributed new reagents tools; and  wrote the paper
;  ; ; ; ;;  designed research;  performed research;  ;  developed the coating;   contributed new reagents tools; ;  wrote the paper
designed research;performed research;developed the coating;contributed new reagents tools;wrote the paper


In [15]:
callback = lambda pat: pat.group(1).upper()+'.'
version17 = version15.apply(lambda x: re.sub('([a-z])\.(?!$)', callback,str(x)))
version17 = version17.apply(lambda x: re.sub(',',';',str(x)))
version17 = version17.apply(lambda x: re.sub('[A-Z]\.|[A-Z]\w*','',str(x)))
version17 = version17.apply(lambda x: re.sub('[^a-zA-Z;\s]\w*','',str(x)))
version17 = version17.apply(lambda x: re.sub('and',';',str(x)))
version17 = version17.str.lstrip()
version17 = version17.apply(lambda x: re.sub('\\s+',' ',str(x)))
version17 = version17.apply(lambda x: re.sub('(;\s)+',';',str(x)))
version17 = version17.apply(lambda x: re.sub('\.$','',str(x)))
version17 = version17.apply(lambda x: re.sub('^;*','',str(x)))

In [16]:
version17.sample(5)

5676     designed research;performed research;analyzed ...
36210    designed research;performed research;contribut...
29711    designed research;performed research;contribut...
39306    designed research;performed research;contribut...
30635    designed research;performed research;analyzed ...
Name: Author contributions, dtype: object

#### Extract our cleaned data

Primarily, we want to extract unique author contritions. But we also want to match doi to author contributions for future project variations.

So let us include our results into a dataframe.

In [17]:
df2 = df.copy()

In [18]:
df2['revision_one'] = version17

In [19]:
df3 = df2[df2['revision_one'].notna()]

In [20]:
df3.sample(5)

,VOL,ISS,DOI,Author contributions,revision_one
23423,113,48,10.1073/pnas.1616889113,"<p>Author contributions: R.F., P.K.L., and A.Z...",designed research;performed research;fabricate...
41958,108,45,10.1073/pnas.1115348108,"<p>Author contributions: Y.B., L.Y., and X.Z. ...",designed research;performed research;contribut...
19733,114,48,10.1073/pnas.1706597114,<p>Author contributions: K.A.A. designed resea...,designed research;performed research;analyzed ...
39920,108,13,10.1073/pnas.1007811108,"<p>Author contributions: M.R., M. Armaka, M.M....",designed research;performed research;analyzed ...
29255,111,27,10.1073/pnas.1313071111,<p>Author contributions: K.R.G. and M.T.Y. des...,designed research;performed research;wrote the...


We are going to split the revision two column so that we can see each of the author contributions.

In [21]:
sub_df = df3['revision_one'].str.split(';',expand=True)

In [22]:
sub_df.head()

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
0,designed research,performed research,contributed new reagents tools,analyzed data,wrote the paper,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,designed research,performed research,analyzed data,wrote the paper,contributed to design of analyses,contributed data,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,designed research,performed research,analyzed data,wrote the paper,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,designed research,performed research,contributed new reagents tools,analyzed data,wrote the paper,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,designed research,performed research,contributed new reagents tools,analyzed data,provided the age model,provided laboratory infrastructure,support,collected the sediment core,extracted the model data,ran the climate model,...,None,None,None,None,None,None,None,None,None,None


We can now extract unique contributions. The technique is to convert to a dataframe and save as csv.
We have to use .ravel function to flatten the array into 1D.

In [23]:
unique = pd.DataFrame(pd.unique(sub_df.values.ravel()))
unique= unique.sort_values(0)

In [24]:
unique.head()

,0
106,
4115,a stratigraphic framework
5793,a tool for classifying free affiliations
6066,absolute quantification assay
941,abundance analysis shown in toggle


In [25]:
unique.sample(10)

,0
2814,data visualization
4390,immunostaining
4654,assays
2284,collected the patient samples
5382,developed image reconstruction
7390,ophthalmic eye examinations in dog
5643,coordinated cross researchers
7844,supported the statistical analysis
5370,implemented temporal models for the f
126,developed the device


In [26]:
unique.shape

(8551, 1)

In [27]:
unique.to_csv('extract.csv')

#### Contact Me!
Thank you for reading this! Let me know if you have any suggestions I can use to make my code better!

#### Next Steps

We have 8k+ unique contributions. We are going to group them into 5 contribution groups, and we're going to attempt to count how many authors worked on each group per journal.